# Load Packages

In [2]:
# !pip install monai --no-index --find-links=file:///kaggle/input/neckfracpackages/monai
# !pip install pydicom --no-index --find-links=file:///kaggle/input/neckfracpackages/pydicom
# !pip install python-gdcm --no-index --find-links=file:///kaggle/input/neckfracpackages/pydicom
# !pip install pylibjpeg --no-index --find-links=file:///kaggle/input/neckfracpackages/pydicom
# !pip install torchvision --no-index --find-links=file:///kaggle/input/neckfracpackages/torchvision

Looking in links: file:///kaggle/input/neckfracpackages/monai
Looking in links: file:///kaggle/input/neckfracpackages/pydicom
Looking in links: file:///kaggle/input/neckfracpackages/pydicom
Looking in links: file:///kaggle/input/neckfracpackages/pydicom
Looking in links: file:///kaggle/input/neckfracpackages/torchvision


In [3]:
!pip install warmup_scheduler
!pip install monai
!pip install -U "python-gdcm" pydicom pylibjpeg
!pip install -U torchvision
!pip install opencv-python
!pip install albumentations

In [4]:
# Libraries
import os
import re
import gc
import cv2
import wandb
import PIL
from PIL import Image
from sklearn.metrics import classification_report
import random
import math
import shutil
from glob import glob
from tqdm import tqdm
from pprint import pprint
from time import time
import warnings
import itertools
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.patches import Rectangle
from IPython.display import display_html
plt.rcParams.update({'font.size': 16})

# .dcm handling
import pydicom
# import nibabel as nib
from pydicom.pixel_data_handlers.util import apply_voi_lut

# Environment check
warnings.filterwarnings("ignore")

In [5]:
# PyTorch
import torch
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR
import torchvision 
import torchvision.transforms as transforms
# from warmup_scheduler import GradualWarmupScheduler
import albumentations

from sklearn.model_selection import GroupKFold, train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score, cohen_kappa_score, confusion_matrix

# MONAI 3D
from monai.transforms import Randomizable, apply_transform
from monai.transforms import Compose, Resize, ScaleIntensity, ToTensor, RandAffine
from monai.networks.nets import densenet

### Helper Function

In [6]:
def read_data():
    '''Reads in all .csv files.'''
    
    train = pd.read_csv(f"{INPUT}/train.csv")
    train_bbox = pd.read_csv(f"{INPUT}/train_bounding_boxes.csv")
    test = pd.read_csv(f"{INPUT}//test.csv")
    # ss = pd.read_csv(f"{INPUT}//sample_submission.csv")
    
    return train, train_bbox, test#, ss

def get_csv_info(csv, name="Default"):
    '''Prints main information for the speciffied .csv file.'''
    
    print(f"=== {name} ===")
    print(f"Shape:", csv.shape)
    print(f"Missing Values:", csv.isna().sum().sum(), "total missing datapoints.")
    print("Columns:", list(csv.columns), "\n")
    
    display_html(csv.head())
    print("\n")
    
def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)  
    torch.cuda.manual_seed(seed)  
    torch.cuda.manual_seed_all(seed)  
    torch.backends.cudnn.deterministic = True
    
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

# Configure

In [7]:
# Environment check
warnings.filterwarnings("ignore")
# os.environ["WANDB_SILENT"] = "true"
CONFIG = {'competition': 'RSNA_SpineFracture', '_wandb_kernel': 'aot'}

# set seed
set_seed(0)

# set GPU
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", DEVICE)

# Kaggle Notebook Setup
DF_SIZE = 0.01
N_SPLITS = 5
KERNEL_TYPE = 'densenet121_baseline'
IMG_RESIZE = 100
STACK_RESIZE = 50
use_amp = False
NUM_WORKERS = 1
BATCH_SIZE = 2
LR = 0.05
OUT_DIM = 8
EPOCHS = 2

target_cols = ['C1', 'C2', 'C3', 
               'C4', 'C5', 'C6', 'C7',
               'patient_overall']

competition_weights = {
    '-' : torch.tensor([1, 1, 1, 1, 1, 1, 1, 7], dtype=torch.float, device=DEVICE),
    '+' : torch.tensor([2, 2, 2, 2, 2, 2, 2, 14], dtype=torch.float, device=DEVICE),
}

Device: cuda


In [18]:
INPUT = "../input/rsna-2022-cervical-spine-fracture-detection"

train, train_bbox, test_df = read_data()

In [56]:
# test RSNA dataset

class RSNADataset_test(Dataset, Randomizable):
    
    def __init__(self, csv, mode, transform=None):
        self.csv = csv
        self.mode = mode
        self.transform = transform
        
    def __len__(self):
        return self.csv.shape[0]
    
    def randomize(self) -> None:
        '''-> None is a type annotation for the function that states 
        that this function returns None.'''
        
        MAX_SEED = np.iinfo(np.uint32).max + 1
        self.seed = self.R.randint(MAX_SEED, dtype="uint32")
        
    def __getitem__(self, index):
        # Set Random Seed
        self.randomize()
        
        dt = self.csv.iloc[index, :]
        study_paths = glob(f"{INPUT}/train_images/{dt.StudyInstanceUID}/*")
        study_paths.sort(key=natural_keys
        
        # Load images
        study_images = [cv2.imread(path)[:,:,::-1] for path in study_paths]
        # Stack all scans into 1
        stacked_image = np.stack([img.astype(np.float32) for img in study_images], 
                                 axis=2).transpose(3,0,1,2)
        
        # Apply transforms
        if self.transform:
            if isinstance(self.transform, Randomizable):
                self.transform.set_random_state(seed=self.seed)
                
            stacked_image = apply_transform(self.transform, stacked_image)
        
        # Determine output of dataset
        if self.mode=="test":
            return {"image": stacked_image,
                   "patient":dt.StudyInstanceUID}
        else:
            targets = torch.tensor(dt[target_cols]).float()
            return {"image": stacked_image,
                    "targets": targets}

In [57]:
# send the data to GPU
def data_to_device(data):
    image, patient = data.values()
    return image.to(DEVICE), patient

In [58]:
# transform
test_transforms = Compose([ScaleIntensity(), 
                          Resize((IMG_RESIZE, IMG_RESIZE, STACK_RESIZE)), 
                          ToTensor()])

In [59]:
# Instantiate Dataset object
test_dataset = RSNADataset_test(csv=train, mode="test", transform=test_transforms)
# The Dataloader
test_dataloader = DataLoader(test_dataset, batch_size=3, shuffle=False)

# Load trained model

In [24]:
model = densenet.densenet121(spatial_dims=3, in_channels=3,
                                 out_channels=OUT_DIM)
model.class_layers.out = nn.Sequential(nn.Linear(in_features=1024, out_features=OUT_DIM), 
                                           nn.Softmax(dim=1))
model.to(DEVICE)

DenseNet121(
  (features): Sequential(
    (conv0): Conv3d(3, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
    (norm0): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (layers): Sequential(
          (norm1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv3d(64, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (norm2): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv3d(128, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        )
      )
      (denselayer2): _DenseLayer(
        (layers): Sequential(
 

In [25]:
# Load checkpoint
PATH='./densenet121_baseline_best_fold0.pth'
if torch.cuda.is_available():
    checkpoint = torch.load(PATH)
else:
    checkpoint = torch.load(PATH, map_location=torch.device('cpu'))

# Load states

model.load_state_dict(checkpoint)

# Evaluation mode
model.eval()
model.to(DEVICE)

DenseNet121(
  (features): Sequential(
    (conv0): Conv3d(3, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
    (norm0): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (layers): Sequential(
          (norm1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv3d(64, 128, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (norm2): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv3d(128, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        )
      )
      (denselayer2): _DenseLayer(
        (layers): Sequential(
 

# Inference

In [ ]:
test_preds = []
for i, data in enumerate(test_dataloader):
        print(f'Iteration {i+1}/{len(test_dataloader)}')
        # Send to device
        imgs, patient = data_to_device(data)
        
        # Make predictions
        preds = model(imgs)
        
        # output to device
        test_preds.append(preds.detach().cpu().numpy())

In [ ]:
preds = []
for i in test_preds:
    for j in i:
        preds.append(j)

In [ ]:
def proccess_test(df, preds):
    cols = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'patient_overall']
    patients = df.StudyInstanceUID.to_list()
    
    df_sub = pd.DataFrame()
    
    for i, p in enumerate(patients):
        scores = list(preds[i])
        if len(scores) < 8:
            scores.append(preds[i].max() + preds[i].mean())
        
        df_temp = pd.DataFrame({'StudyInstanceUID': [p]*len(cols), 'prediction_type': cols, 'fractured': scores})
        df_sub = pd.concat([df_sub, df_temp])
        
        del df_temp
    
    df_sub['row_id'] = df_sub['StudyInstanceUID'] + '_' + df_sub['prediction_type']
    
    return df_sub[['row_id', 'fractured']].reset_index(drop = True)
    

In [ ]:
df_sub = process_test(train,preds)

In [ ]:
train.melt(id_vars=['Study'])

In [ ]:
df_sub = proccess_test(test_df, preds)
df_sub.to_csv('submission.csv', index=False)
df_sub